In [ ]:
#QIKSIT TRAINING

!pip install qiskit
!pip install qiskit_machine_learning
from qiskit import QuantumCircuit, assemble, Aer
from qiskit_machine_learning.algorithms.classifiers import VQC
import qiskit
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import  RealAmplitudes, EfficientSU2, TwoLocal, NLocal
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, ADAM

In [ ]:
def FeatureMap():
  num_qubits = 4
  x = ParameterVector('x', length = num_qubits)
  feature_map = QuantumCircuit(num_qubits)

  for i in range(num_qubits):
    feature_map.rx(x[i],i)
    feature_map.h(i)
  return feature_map

In [ ]:
from qiskit.circuit.library.n_local import real_amplitudes
def VarCirc():
  num_qubits = 4
  #var_circuit = TwoLocal(4, ['rx', 'ry','rz'], 'cx', entanglement = 'circular', reps = 1, insert_barriers = True)
  reps = 3

  psi = ParameterVector('psi', length = num_qubits*reps*3)
  var_circuit = QuantumCircuit(num_qubits)

  for j in range(reps):
    for i in range(num_qubits-1):
      var_circuit.cx(i,i+1)
    var_circuit.cx(num_qubits-1, 0)

    for i in range(num_qubits):
      var_circuit.rx(psi[3*i + 0 + num_qubits*3*j], i)
      var_circuit.ry(psi[3*i + 1 + num_qubits*3*j], i)
      var_circuit.rz(psi[3*i + 2 + num_qubits*3*j], i)

  return var_circuit

In [ ]:
import torch.nn as nn
import math
class TNLayer(nn.Module):
    def __init__(self):
        super(TNLayer, self).__init__()
        # Create the variables for the layer.
        self.a_var = nn.Parameter(torch.randn(28, 28, 2) * (1.0 / 28.0))
        self.b_var = nn.Parameter(torch.randn(28, 28, 2) * (1.0 / 28.0))
        self.bias = nn.Parameter(torch.zeros(28, 28))

    def forward(self, inputs):
        # Define the contraction.
        def f(input_vec, a_var, b_var, bias_var):
            # Reshape to a matrix instead of a vector.
            input_vec = input_vec.reshape(28, 28)
             #input_vec = input_vec.reshape(2, 512)

            # Now we create the network.
            a = a_var.unsqueeze(0)  # Shape: (1, 32, 32, 2)
            b = b_var.unsqueeze(0)  # Shape: (1, 32, 32, 2)
            x_node = input_vec.unsqueeze(-1)  # Shape: (32, 32, 1)

            # Contract a with x_node
            c = torch.einsum("ijkl,ijm->ijkm", a, x_node)  # Shape: (32, 32, 32, 2)

            # Contract the result with b
            result = torch.einsum("ijkl,ijkl->ij", c, b)  # Shape: (32, 32)

            # Finally, add bias.
            return result + bias_var

        # To deal with a batch of items, loop through the input batch.
        results = []
        for input_vec in inputs:
            results.append(f(input_vec, self.a_var, self.b_var, self.bias))
        return nn.functional.relu(torch.stack(results))



class VQC(nn.Module):
  def __init__(self, n, num_layers):
    super(VQC, self).__init__()
    self.n = n
    self.num_layers = num_layers
    self.params = nn.Parameter(torch.randn(self.n, 3 * self.num_layers))

  def prints(self, psi):
    print("-------------------------MPS STRUCTURE SHAPE------------------------------------------")
    for _, i in enumerate(psi):
      print(i.shape)
    print("-------------------------SHAPE------------------------------------------")

  def H(self, i, psi):
    H_matrix = 1 / ((2) ** 0.5) * torch.tensor([[1, 1],
                                                 [1, -1]], dtype=torch.cfloat)
    psi[i] = torch.tensordot(H_matrix, psi[i], ([1], [2]))
    psi[i] = torch.moveaxis(psi[i], 0, 2)
    return psi

  def RX(self, i, j, psi):  # Alter the array psi into the array where H has acted on the ith qubit
    t1 = (torch.cos(i / 2)).reshape(1)
    t2=(torch.sin(i/2)*-1j).reshape(1)
    t3=(torch.sin(i/2)*-1j).reshape(1)
    t4=(torch.cos(i/2)).reshape(1)
    f1=torch.cat([t1,t2])
    f2=torch.cat([t3,t4])
    rx=torch.cat([f1,f2]).reshape(2,2).type(torch.cfloat)
    psi[j]=torch.tensordot(rx,psi[j],([1],[2]))
    psi[j]=torch.moveaxis(psi[j],0,2)
    return psi
  def RY(self,i,j,psi): # Alter the array psi into the array where H has acted on the ith qubit
    t1=(torch.cos(i/2)).reshape(1)
    t2=-1.*(torch.sin(i/2)).reshape(1)
    t3=(torch.sin(i/2)).reshape(1)
    t4=(torch.cos(i/2)).reshape(1)
    f1=torch.cat([t1,t2])
    f2=torch.cat([t3,t4])
    ry=torch.cat([f1,f2]).reshape(2,2).type(torch.cfloat)
    psi[j]=torch.tensordot(ry,psi[j],([1],[2]))
    psi[j]=torch.moveaxis(psi[j],0,2)
    return psi

  def RZ(self,i,j,psi): # Alter the array psi into the array where H has acted on the ith qubit
    t1=(torch.exp(-1j*i/2)).reshape(1)
    t2=(torch.zeros(1)).reshape(1)
    t3=(torch.zeros(1)).reshape(1)
    t4=(torch.exp(1j*i/2)).reshape(1)
    f1=torch.cat([t1,t2])
    f2=torch.cat([t3,t4])
    rz=torch.cat([f1,f2]).reshape(2,2).type(torch.cfloat)
    psi[j]=torch.tensordot(rz,psi[j],([1],[2]))
    psi[j]=torch.moveaxis(psi[j],0,2)
    return psi

  def CNOT(self, i, j, psi):
     dim = psi[i].shape[0]
     mps=torch.tensordot(psi[i],psi[j],([1],[0]))
     mps=torch.moveaxis(mps,2,1)
     CNOT_matrix=torch.tensor([[1,0,0,0],
                       [0,1,0,0],
                       [0,0,0,1],
                       [0,0,1,0]],dtype=torch.cfloat)
     CNOT_tensor=torch.reshape(CNOT_matrix, (2,2,2,2))
     mps=torch.tensordot(CNOT_tensor, mps, ([2,3],[2, 3]))
     mps=torch.moveaxis(mps,1,3).reshape((2*dim,2*dim))
     #mps+=mps+(torch.randn(mps.shape))*0.001
     data=torch.rand(mps.shape)
     data=data*mps.abs().max()*0.000001/data.max()
     mps=mps+data
     #u,s,v = torch.linalg.svd(mps)
     u,s,v = torch.svd(mps)
     v = torch.conj(v.T)
     c = torch.diag(s[:dim])
     mps1 = torch.mm(u[:,:dim], c.type(torch.cfloat))
     mps1 = mps1.reshape((2,dim, dim))
     psi[i] = torch.moveaxis(mps1,0,2)
     mps2 = v[:dim,:]
     psi[j] = mps2.reshape((dim, dim,2))
     return psi

  def CNOT_full(self,i,j,psi):
        # Alter the array psi into the array where H has acted on the ith qubit
        #i-->control;
        #j-->target;
        s0=psi[i].shape[0]
        s1=psi[j].shape[1]
        # print(i,j)
        # print(psi[i].shape)
        # print(psi[j].shape)
        mps=torch.tensordot(psi[i],psi[j],([1],[0]))
        mps=torch.moveaxis(mps,2,1)
        #print(mps.shape)
        CNOT_matrix=torch.tensor([[1,0,0,0],
                      [0,1,0,0],
                      [0,0,0,1],
                      [0,0,1,0]],dtype=torch.cfloat)

        CNOT_tensor=torch.reshape(CNOT_matrix, (2,2,2,2))
        mps=torch.tensordot(CNOT_tensor, mps, ([2,3],[2,3]))
        mps=torch.moveaxis(mps,1,3).reshape((s0*2,s1*2))
        data=torch.rand(mps.shape)
        data=data*mps.abs().max()*0.00001/data.max()
        mps=mps+data
        u, s, v =  torch.linalg.svd(mps)
        c=(torch.zeros(u.shape[1],v.shape[0]))
        for h,k in enumerate(s):
          c[h,h]=k
        mps1=torch.mm(u, c.type(torch.cfloat))
        mps1=mps1.reshape((2,int(mps1.shape[0]/2),mps1.shape[1]))
        psi[i]=torch.moveaxis(mps1,0,2)
        mps2=v#.transpose(-2, -1)
        psi[j]=mps2.reshape((mps2.shape[0],int(mps2.shape[1]/2),2))
        return psi

  def getParams(self):
    return self.params

  def setParams(self, params_matrix):
    self.params = params_matrix

  def compute_tensor_circuit(self, psi, t):
    #Embedding layer
    for i in range(self.n):
      self.RX(t[i], i, psi)
      self.H(i, psi)

    # Parametrized Layers
    #Each layer contains a series of parametrized RX, RY, RZ gates and cyclic entanglement
    for i in range(self.num_layers):
      #CNOTs
      for j in range(self.n-1):
        psi = self.CNOT(j, j+1, psi)
      psi = self.CNOT(self.n-1,0, psi)

      #Parametrized rotations
      for j in range(self.n):
        psi =self.RX(self.params[j][3*i + 0], j, psi)
        psi = self.RY(self.params[j][3*i + 1], j, psi)
        psi = self.RZ(self.params[j][3*i + 2], j, psi)

    return psi

  def contract_tensor_ring(self,psi):
    psi_new = psi[0]
    for i in range(self.n-2):
      psi_new = torch.tensordot(psi_new, psi[i+1], ([1], [0]))
      psi_new = torch.moveaxis(psi_new, -2, 1)

    psi_new = torch.tensordot(psi_new, psi[self.n-1], ([0,1], [1,0]))
    return psi_new

  # def FeatureMap():
  #   num_qubits = 4
  #   x = ParameterVector('x', length = num_qubits)
  #   feature_map = QuantumCircuit(num_qubits)

  #   for i in range(num_qubits):
  #      feature_map.rx(x[i],i)
  #      feature_map.h(i)
  #   return feature_map

  # def VarCirc():
  #    num_qubits = 4
  # #var_circuit = TwoLocal(4, ['rx', 'ry','rz'], 'cx', entanglement = 'circular', reps = 1, insert_barriers = True)
  #    reps = 3

  #    psi = ParameterVector('psi', length = num_qubits*reps*3)
  #    var_circuit = QuantumCircuit(num_qubits)

  #    for j in range(reps):
  #     for i in range(num_qubits-1):
  #        var_circuit.cx(i,i+1)
  #        var_circuit.cx(num_qubits-1, 0)

  #    for i in range(num_qubits):
  #     var_circuit.rx(psi[3*i + 0 + num_qubits*3*j], i)
  #     var_circuit.ry(psi[3*i + 1 + num_qubits*3*j], i)
  #     var_circuit.rz(psi[3*i + 2 + num_qubits*3*j], i)

  #    return var_circuit


  def forward(self, psi, t):

    # Convert tensor t to complex tensor
    # print(t.size())
    # t_complex = torch.view_as_complex(t)
    # print(t_complex.size())
    psi_final = self.compute_tensor_circuit(psi, t)
    k = self.contract_tensor_ring(psi_final)
    k1 = k[0, 0, 0, 0].abs().reshape(1) ** 2
    k2 = k[0, 0, 1, 0].abs().reshape(1) ** 2
    k3 = k[1, 1, 1, 1].abs().reshape(1) ** 2

    return torch.cat([k1, k2, k3]).reshape(1, 3)

  def final_state(self, psi,t):

    #psi_final = self.test_circuit(psi,t)
    psi_final = self.compute_tensor_circuit(psi, t)
    k = self.contract_tensor_ring(psi_final)
    return k

  def create_zero_tensor(num_qubits, d):
    psi_0 = []
    for j in range(num_qubits):
      k = torch.zeros((d,d,2), dtype=torch.cfloat)
      k[0,0,0] = 1.0
      psi_0.append(k)
    return psi_0

